# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called metastatemnt or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

### Input variables

In [74]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  code, header_in_row
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5) # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0
}

table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with schemas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [75]:
import subprocess
subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

CompletedProcess(args='python generate-json-schemas.py', returncode=0, stdout=b'typescript-json-schema ../../shared/types/actant.ts IActant --required --noExtraProps --defaultProps --id IActant --out ../schemas/IActant.schema.json\r\ntypescript-json-schema ../../shared/types/concept.ts IConcept --required --noExtraProps --defaultProps --id IConcept --out ../schemas/IConcept.schema.json\r\ntypescript-json-schema ../../shared/types/object.ts IObject --required --noExtraProps --defaultProps --id IObject --out ../schemas/IObject.schema.json\r\ntypescript-json-schema ../../shared/types/event.ts IEvent --required --noExtraProps --defaultProps --id IEvent --out ../schemas/IEvent.schema.json\r\ntypescript-json-schema ../../shared/types/entity.ts IEntity --required --noExtraProps --defaultProps --id IEntity --out ../schemas/IEntity.schema.json\r\ntypescript-json-schema ../../shared/types/label.ts ILabel --required --noExtraProps --defaultProps --id ILabel --out ../schemas/ILabel.schema.json\r\n

### Libraries

In [76]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [77]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-04-10 10:28:32 : Google authentification start
20 2022-04-10 10:28:32 : Google authentification end
20 2022-04-10 10:28:32 : Dator initiation succesfull end


In [78]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-04-10 10:28:32,360 INFO Class IActant available.
2022-04-10 10:28:32,369 INFO Class IAction available.
2022-04-10 10:28:32,375 INFO Class IAudit available.
2022-04-10 10:28:32,383 INFO Class IConcept available.
2022-04-10 10:28:32,392 INFO Class IEntity available.
2022-04-10 10:28:32,398 INFO Class IEvent available.
2022-04-10 10:28:32,404 INFO Class ILabel available.
2022-04-10 10:28:32,412 INFO Class ILocation available.
2022-04-10 10:28:32,418 INFO Class IObject available.
2022-04-10 10:28:32,423 INFO Class IProp available.
2022-04-10 10:28:32,429 INFO Class IReference available.
2022-04-10 10:28:32,435 INFO Class IResource available.
2022-04-10 10:28:32,440 INFO Class IStatement available.
2022-04-10 10:28:32,447 INFO Class ITerritory available.
2022-04-10 10:28:32,453 INFO Class IUser available.
2022-04-10 10:28:32,459 INFO Class IValue available.
2022-04-10 10:28:32,460 INFO There are 16 json classes available (IActant IAction IAudit IConcept IEntity IEvent ILabel ILocation 

In [79]:
# factory for making entity objects, contains defaults with "prerequisities"
from datetime import datetime
import_note = "Import batch [development] " + str(datetime.now())

class InkVisitorJSONObjectFactory:

    classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'','legacyId':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''}}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IValue':{
            'class':'V', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'4'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'1', 'children':[], 'elvl':'1',  'id':'', 'logic':'1', 'mood':[], 'moodvariant':'1', 'bundleOperator':'=', 'type': {'id':'','elvl':'1','logic':'1','partitivity':'1','virtuality':'1'},'value':{'id':'','elvl':'1', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'url':'','partValueBaseURL':'','partValueLabel':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IObject':{
             'class':'O', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'tags':[],'territory': {'id':'','order':0}, 'text':''}, 'props':[], 'notes':[], 'status':'1','references':[]
        },
        'ITerritory':{
            'class':'T', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{'parent':{ "id": "T0", "order": 0 }}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'ILocation':{
            'class':'L', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IEvent':{
            'class':'E', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IReference':{
           'id':'','resource':'','value':''
        },
        'IAudit':{
           'id':'','entityId':'','user':'','date':'','changes':{}
        }
    }

    def __init__(self):
        for key, item in type(self).json_class_defaults.items():
            if 'notes' in item and  len(item['notes']) == 0:
                item['notes'].append(import_note)

    def make(self, entity_name, override_object=None):
        if override_object is None:
            override_object = {}
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IOF = InkVisitorJSONObjectFactory()
IOF.validate_defaults()

2022-04-10 10:28:32,503 INFO Class IAction validated.
2022-04-10 10:28:32,509 INFO Class IConcept validated.
2022-04-10 10:28:32,513 INFO Class IValue validated.
2022-04-10 10:28:32,518 INFO Class IProp validated.
2022-04-10 10:28:32,523 INFO Class IResource validated.
2022-04-10 10:28:32,528 INFO Class IObject validated.
2022-04-10 10:28:32,533 INFO Class IStatement validated.
2022-04-10 10:28:32,538 INFO Class ITerritory validated.
2022-04-10 10:28:32,543 INFO Class ILocation validated.
2022-04-10 10:28:32,548 INFO Class IEvent validated.
2022-04-10 10:28:32,549 INFO Class IReference validated.
2022-04-10 10:28:32,551 INFO Class IAudit validated.


# Load input datasets

In [156]:
# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ['NA', "#N/A", "#VALUE!"]
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value)
    df. fillna(unified_empty_value, inplace=True)
    return df



# load all input tables
tables = {}
header_infos = {}
entity_ids = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=False, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # code for legacyId copy and uuid creation
    # tables[key]['legacyId'] = tables[key]['id'].copy()
    tables[key].insert(2, 'legacyId', tables[key]['id'].copy())

    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")
    entity_ids[key] = ed["id"].to_dict()

2022-04-10 15:35:45,005 INFO Calling for texts with sheet_name Texts.


20 2022-04-10 15:35:45 : Loading dataset Texts
20 2022-04-10 15:35:45 : Opting for variant header at row 5.
20 2022-04-10 15:35:50 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-04-10 15:35:50 : Dropping empty columns in the dataset Texts : (1011, 116)


2022-04-10 15:35:50,375 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-04-10 15:35:50 : Deleted 869 empty rows by label.
20 2022-04-10 15:35:50 : Loaded and prepared dataset Texts : (142, 116)
20 2022-04-10 15:35:50 : Making pickle cache of  Texts with separeted header file : (142, 116)
20 2022-04-10 15:35:50 : Loading dataset Manuscripts
20 2022-04-10 15:35:50 : Opting for variant header at row 5.


2022-04-10 15:35:52,867 INFO Calling for resources with sheet_name Resources.


20 2022-04-10 15:35:52 : Hyperlinks were detected and transformed in columns ['reproduction_online_url', 'reproduction_note'].
20 2022-04-10 15:35:52 : Dropping empty columns in the dataset Manuscripts : (1036, 65)
20 2022-04-10 15:35:52 : Deleted 897 empty rows by label.
20 2022-04-10 15:35:52 : Loaded and prepared dataset Manuscripts : (139, 65)
20 2022-04-10 15:35:52 : Making pickle cache of  Manuscripts with separeted header file : (139, 65)
20 2022-04-10 15:35:53 : Loading dataset Resources
20 2022-04-10 15:35:53 : Opting for variant header at row 5.


2022-04-10 15:35:54,590 INFO Calling for actions with sheet_name Statements.


20 2022-04-10 15:35:54 : Hyperlinks were detected and transformed in columns [].
20 2022-04-10 15:35:54 : Dropping empty columns in the dataset Resources : (1000, 22)
20 2022-04-10 15:35:54 : Deleted 928 empty rows by label.
20 2022-04-10 15:35:54 : Loaded and prepared dataset Resources : (72, 22)
20 2022-04-10 15:35:54 : Making pickle cache of  Resources with separeted header file : (72, 22)
20 2022-04-10 15:35:55 : Loading dataset Statements
20 2022-04-10 15:35:55 : Opting for variant header at row 5.
20 2022-04-10 15:36:09 : Hyperlinks were detected and transformed in columns [].
20 2022-04-10 15:36:09 : Dropping empty columns in the dataset Statements : (1030, 77)
20 2022-04-10 15:36:09 : Deleted 588 empty rows by label.
20 2022-04-10 15:36:09 : Loaded and prepared dataset Statements : (442, 77)
20 2022-04-10 15:36:09 : Making pickle cache of  Statements with separeted header file : (442, 77)


2022-04-10 15:36:09,895 INFO Calling for concepts with sheet_name Concepts.


20 2022-04-10 15:36:10 : Loading dataset Concepts
20 2022-04-10 15:36:10 : Opting for variant header at row 5.
20 2022-04-10 15:36:13 : Hyperlinks were detected and transformed in columns [].
20 2022-04-10 15:36:13 : Dropping empty columns in the dataset Concepts : (3029, 66)
20 2022-04-10 15:36:14 : Deleted 224 empty rows by label.
20 2022-04-10 15:36:14 : Loaded and prepared dataset Concepts : (2805, 66)
20 2022-04-10 15:36:14 : Making pickle cache of  Concepts with separeted header file : (2805, 66)


In [103]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])

In [121]:
header_infos['texts']

,id,label,label_language,label_short,text_name_original,detail,region_covered,region_covered_id,region_covered_label,microregion_covered,...,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,editor,parsing_rows_explained,legacyId
0,,,,,Now concatenated with detail. Creating a new T...,,,,,,...,,,,,,,,audits,Comment - sometimes important to read for pars...,
1,,,,,,,,,,,...,,,,,,,,,Source node (for props),
2,,,,,,,,,,,...,,,,,,,note,,Prop type (for props) or further detail (for i...,
3,inside,inside,inside,discard,discard,inside,discard,discard,discard,discard,...,discard,discard,discard,discard,discard,discard,inside,special,discard,inside


In [123]:
tables['texts']

,id,label,legacyId,label_language,label_short,text_name_original,detail,region_covered,region_covered_id,region_covered_label,...,dissinet_person,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,editor,parsing_rows_explained
0,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,Process against Bernard Niort and his family,T1,English,,,Early 1234.,Languedoc,L0015,Languedoc,...,,,,,,,deposition,,,
1,3f6dc058-83a6-407c-b4ce-de96304ff6f7,Sentences of William Arnold and Stephen of Sai...,T2,English,,,,Languedoc,L0015,Languedoc,...,RS?,,,,,register,sentence,End-folio sometimes cited as 184v (e.g. Roche...,,
2,9d17a658-d954-4e3d-bb7b-08067a7e307a,Peter Seila’s Register of Penances,T3,English,Seila,Penitenciae fratris Petri Sellani,Penitenciae fratris Petri Sellani. Register of...,Languedoc,L0015,Languedoc,...,RS,,,,,register,sentence #culpa,,,
3,17bac2e7-74a2-4be6-88a3-ec02f3546004,Register FFF of the Carcassonne inquisition,T4,English,FFF,,,Languedoc,L0015,Languedoc,...,,,,,,register,deposition,,,
4,c694f534-3166-4e15-9a81-0aa96287ac37,Confirmation of depositions before Ferrer and ...,T5,English,,,,Languedoc,L0015,Languedoc,...,,,,,,register,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,d4da2c3b-dc69-41b3-84b7-d3d9cef58440,letter of Evervin of Steinfeld to Bernard of C...,T139,English,,,,,,,...,,,,,,,,,,
138,9ea4b86f-b3a9-4b6d-bc53-0b07617dde26,letter from Liège to pope,T140,English,,,,,,,...,,,,,,,,,,
139,19b7393d-e95e-4775-bb37-14eb76c4e1ec,Annales Aquenses,T141,Latin,,,,,,,...,,,,,,,,,,
140,f2bfb8af-c2f7-49d0-8248-5d39dec6950a,Annales Rodenses,T142,Latin,,,,,,,...,,,,,,,,,,


### Responsible editors

#### Data
Only concepts have "editors" filled.

#### Specification

Only entities have audits, i.e. the prop object, the ref object don't. (Could be wrong?)

**Rationale:**  Each entity will have 2 audit creation records. One with the user "import", second with the user "responsible editor.

Implementaions possiblities:
  - make the audits during the parsing : the way
  - make the audits afer the parsing is done (for secondary created entities, I will not have the context, and will be problem to find an responsible editor
  -

In [84]:
tables['concepts']['editor'].value_counts()

                   1261
David Zbíral        865
Katia Riccardo      266
Davor Salihović     245
Robert Shaw         168
Name: editor, dtype: int64

In [85]:
# the names in the input tables  vs their user ids
# the tuser are imported through the user.json file

editors = {"import":"0", "David Zbíral":"100", "Robert Shaw":"101", "Davor Salihović":"102", "Katia Riccardo":"103"}

# Declaration of controlling classes

In [153]:
additional_entities = []
audits = []

# for controlling entity and mapping of its fields
class EntityMapper:

    # simple inside values mapping from input_values in gsheets to inkVisitor enums
    # field: { FROM  : TO }
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci","Middle English":"enm","Czech":"ces","Italian":"ita","French":"fra","German":"deu"},"status":{"approved":"1","pending":"0","discouraged":"2","warning":"3"}}
    # status  Pending = "0",   Approved = "1",  Discouraged = "2",  Warning = "3",
    valid_entity_classes = ['A','C','E','O','R','T','P','G','S','L','NULL']


    IOF = InkVisitorJSONObjectFactory()

    def __init__(self, entity_type, data_row, logger = d.logger):
        self.entity =  type(self).IOF.make(entity_type)
        self.logger = logger
        self.debug = True
        self.data_row = data_row


    def make_prop_object(self,prop_type_id, prop_value_id):
        prop_object = IOF.make('IProp')
        prop_object['id'] = get_uuid_id()
        prop_object['type']['id'] = prop_type_id
        prop_object['value']['id'] = prop_value_id
        return prop_object

    def make_ref_object(self, ref_resource_id, ref_value_id):
        ref_object = IOF.make('IReference')
        ref_object['id'] = get_uuid_id()
        ref_object['resource'] = ref_resource_id
        ref_object['value'] = ref_value_id
        return ref_object

    def get_entity_id(self,entity_string, origin = ""):
        id = ""
        # logger.info(f"Getting entity string {entity_string} in {origin}")
        try:
            if entity_string.startswith("C") and entity_string[1:].isnumeric():
                # id = tables['concepts'].loc[tables['concepts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["concepts"][entity_string]
            elif entity_string.startswith("~V~"):
                ventity = self.make_ventity(entity_string, origin=origin)
                id = ventity['id']
            elif entity_string.startswith("M") and entity_string[1:].isnumeric():
                # id = tables['manuscripts'].loc[tables['manuscripts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["manuscripts"][entity_string]
            elif entity_string.startswith("A") and entity_string[1:].isnumeric():
                # id = tables['actions'].loc[tables['actions']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["actions"][entity_string]
            elif entity_string.startswith("R") and entity_string[1:].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["resources"][entity_string]
            elif entity_string.startswith("T") and entity_string[1:].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["texts"][entity_string]

            elif is_valid_uuid(entity_string):
                id = entity_string

        except IndexError as E:
            logger.info(f"Cannot get entity id from entity string {entity_string} in {origin}. {E}")

        if id != "" and isinstance(id, str):
            # logger.info(f"Got entity id {id} from entity string {entity_string} in {origin}")
            return id
        else:
            logger.error(f"Cannot get entity id from {entity_string}.")
            raise Exception(f"Cannot get entity id from {entity_string}.")

    def make_ventity(self, value_string, origin=""):
        # logger.info(f"Generating ventity from {value_string}.")
        # generate value entity object...
        ventity = IOF.make('IValue')
        ventity['id'] = get_uuid_id()
        ventity['label'] = value_string.replace("~V~","")

        if self.debug:
            ventity['notes'].append(origin)

        # register ventity
        tables['values'] = tables['values'].append({'id':ventity['id'] ,'value':ventity['label'],"origin":origin},ignore_index=True )
        additional_entities.append(ventity)

        # logger.info(f"Ventity id={ventity['id']} generated.")

        # create audit record
        self.create_audit_record(entity_id=ventity['id'], object=ventity)

        return ventity

    def make_rentity(self, label, url = "", origin=""):
        # logger.info(f"Generating rentity from {value_string}.")
        # generate resource entity object...
        rentity = IOF.make('IResource')
        rentity['id'] = get_uuid_id()
        rentity['label'] = label
        rentity['data']['url'] = url

        if self.debug:
            rentity['notes'].append(origin)

        # register rentity
        tables['resources'] = tables['resources'].append({'id':rentity['id'] ,'value':rentity['label'],"origin":origin}, ignore_index=True )
        additional_entities.append(rentity)

        # logger.info(f"Rentity id={rentity['id']} generated.")
        return rentity


    def make_eentity(self, label, legacyId, url = "", origin=""):

        eentity = IOF.make('IEvent')
        eentity['id'] = get_uuid_id()
        eentity['label'] = label
        eentity['legacyId'] = legacyId

        if self.debug:
            eentity['notes'].append(origin)

        # register event
        tables['events'] = tables['events'].append({'id':eentity['id'] ,'value':eentity['label'],"origin":origin, "legacyId":legacyId}, ignore_index=True )
        additional_entities.append(eentity)

        return eentity

    def make_lentity(self, label, legacyId="", url = "", origin=""):

        lentity = IOF.make('ILocation')
        lentity['id'] = get_uuid_id()
        lentity['label'] = label
        lentity['legacyId'] = legacyId

        if self.debug:
            lentity['notes'].append(origin)

        # register lentity
        tables['locations'] = tables['locations'].append({'id':lentity['id'] ,'value':lentity['label'],"origin":origin,"legacyId":legacyId}, ignore_index=True )
        additional_entities.append(lentity)

        return lentity

    # interprets prop_type (should be always concept or resource) and input_value (should be concept or value string)
    # get ids of the prop_type and prop_value (possibly creates and register values object)
    # make iProp object
    # puts iProp object into the entity props property
    def hook_prop_object(self, prop_type, input_value, prop_source="",  origin="", prop_source_id ="", field_name="", prop_source_field = ""):

        # allowed entities in type
        assert "C" in prop_type[0] or "R" in prop_type[0], f"Prop type unknown, C or R string entity expected? {prop_type}, {origin}"

        # allowed entities in input
        allowed_strict_entities = ['C','M','A','E','L','R','S','V','O','T'] # should be followed by numbers
        allowed_free_string_entities = ['~V~']

        # checking input_value
        if not is_valid_uuid(input_value):
            if any(input_value.startswith(c)for c in allowed_strict_entities):
                # check for numbers
                if not input_value[1:].isnumeric():
                    input_value = "~V~"+input_value
            elif not any(input_value.startswith(c)for c in allowed_free_string_entities):
                input_value = "~V~"+input_value

        # TODO this is useless now, because the line above will make ~V~ from everything unknown
        assert any(input_value.startswith(c)  for c in allowed_strict_entities) or any(input_value.startswith(c)  for c in allowed_free_string_entities) or is_valid_uuid(input_value), f"Prop value unknown, C string or V string entity expected, or valid uuid.{input_value}, {origin}"

        prop_type_id = self.get_entity_id(prop_type, origin=origin)
        if not is_valid_uuid(input_value):
            prop_value_id = self.get_entity_id(input_value, origin=origin)
        else:
            prop_value_id = input_value
        # logger.info(f"{prop_type_id}, {prop_value_id}")

        # make IProp object
        prop_object = self.make_prop_object(prop_type_id, prop_value_id)

        # register prop object
        tables['props'] = tables['props'].append({'id':prop_object['id'] , 'type_id':prop_type_id,'value_id':prop_value_id,'type':prop_type,'value':input_value, "original_field":field_name, "origin":origin, 'entityId':self.entity['id'], 'legacyId':self.entity['legacyId']}, ignore_index=True)

        if prop_source_field != "":
            self.hook_2ndprop_into_props(prop_object,prop_source_field = prop_source_field)
        elif prop_source_id !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source_id = prop_source_id)
        elif prop_source !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source = prop_source)

        else:
            # hook directly into the entity object
            self.hook_prop_into_props(prop_object)

    def hook_ref_object(self, ref_legacyID, input_value, prop_source="",  origin=""):

        # allowed entities in ref_legacyID
        assert "R" in ref_legacyID, f"Unknown input, R legacyId expected? {ref_legacyID}, {origin}"

        #modify value, so the value object is created
        input_value = "~V~"+input_value

        ref_resource_id = self.get_entity_id(ref_legacyID, origin=origin)
        ref_value_id = self.get_entity_id(input_value, origin=origin)

        # make IReference object
        ref_object = self.make_ref_object(ref_resource_id, ref_value_id)

        self.hook_ref_into_refs(ref_object)


    def hook_prop_into_props(self,prop_object):
        self.entity['props'].append(prop_object)

    def hook_ref_into_refs(self,ref_object):
        self.entity['references'].append(ref_object)

    def hook_2ndprop_into_props(self, prop_object, prop_source = "", prop_source_id = "", prop_source_field = ""):  # identification by concept id

        # recognition based on the prop_source_field
        if prop_source_field != "":

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                # po['id']
                # find whether this id is registered under the prop_source_field within this entity frame
                candidate_prop_objects = tables['props'][(tables['props']['legacyId'] ==self.entity['legacyId']) & (tables['props']['original_field'] == prop_source_field)]

                logger.info(f"2nprop: {len(candidate_prop_objects)}")
                logger.info(f"2nprop: {len(candidate_prop_objects)} : {candidate_prop_objects.iloc[0]['id']}")

                if len(candidate_prop_objects) > 0:
                    for key, row in candidate_prop_objects.iterrows():

                        for  po in self.entity['props']:
                            if po['id'] == row['id'] and len(po['children']) == 0:
                                logger.info(f"Found prop. {po['id']}. Adding the 2ndprop child.")
                                po['children'].append(prop_object)

                else:
                    logger.error(f"Cannot find the proper prob object record. {prop_object} at {prop_source_field}")

        # recognition based on the type concept of the parent prop object
        if prop_source != "":
            keyId = self.get_entity_id(prop_source)
            assert is_valid_uuid(keyId), f"Cannot recognize entity id from {prop_source}"

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                if po['type']['id'] == keyId and len(po['children']) == 0:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)


        # recognition based od propobject id
        if prop_source_id != "":
            keyId = prop_source_id
            assert is_valid_uuid(keyId), "Not valid uuid {keyId}"

            for count, po in enumerate(self.entity['props']):
                if po['id'] == keyId:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)



    # method invoker for the INSIDE operation with concrete fields
    def update_inside_field(self, field_name, input_value, origin= ""):
        if input_value != '':

            if ("#" in input_value or "~" in input_value) and field_name!= "note" and "https://docs." not in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            update_func = getattr(self, update_operation, self.update_generic)
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value, origin)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_label_language(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.logger.error(f"Unable to set language in {origin}.")
            self.entity['language'] = input_value # will raise error

    def update_status(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['status']:
            self.entity['status'] = type(self).enum_mapper['status'][input_value]
        else:
            self.logger.error(f"Unable to set status in {origin}.")
            self.entity['status'] = input_value # will raise error

    def update_note(self, field_name, input_value, origin = ""):
        #self.logger.info(f"Updating note with {input_value}.")
        if "#" in input_value:
            values = [v.strip() for v in input_value.split("#")]
            for v in values:
                self.entity['notes'].append(v)
        else:
            self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value, origin = ""):
        # self.entity['id'] = input_value
        self.entity['id'] = input_value

    def update_legacyId(self, field_name, input_value, origin = ""):
        # logger.info(f"Trying to set legacyId {type(input_value)}:'{input_value}' {origin}.")
        self.entity['legacyId'] = input_value

    def update_label(self, field_name, input_value, origin = ""):
        self.entity['label'] = input_value

    def update_wordnet_lemma_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_lemma_id NOT IMPLEMENTED ")
        pass

    def update_wordnet_synset_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_synset_id NOT IMPLEMENTED ")
        pass

    def update_generic(self, field_name, input_value, origin = ""):
        self.entity[field_name] = input_value


    def create_audit_record(self, entity_id = '', editor_candidate = '', object = {}):

        data_row = self.data_row
        if entity_id == '':
            entity_id = self.entity['id']
        if object == {}:
            object = self.entity  # should be run at the end of the entity making, to the object is "full"

        # date =  {
        #     "$reql_type$": "TIME",
        #      "epoch_time": time.time(),
        #      "timezone": "+00:00"
        # }
        date =  datetime.now().strftime("%Y-%d-%mT%H:%M:%S")+"Z" #2021-12-05T19:10:15.739Z

        if editor_candidate == '' and 'editor' in data_row.keys():
            editor_candidate = data_row['editor']
        if editor_candidate in editors.keys():
            editor_id = editors[editor_candidate]
        else:
            editor_id = editors['David Zbíral']

        import_audit = IOF.make('IAudit')
        import_audit['id'] = get_uuid_id()
        import_audit['entityId'] = entity_id
        import_audit['user'] =  editors['import']
        import_audit['date'] = date
        import_audit['changes'] = object

        editor_audit = IOF.make('IAudit')
        editor_audit['id'] = get_uuid_id()
        editor_audit['entityId'] = entity_id
        editor_audit['user'] =  editor_id
        editor_audit['date'] = date
        editor_audit['changes'] = object

        # audits.append(import_audit)
        audits.append(editor_audit)


class TerritoryEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ConceptEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ActionEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger,):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_subject_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['s'].append(e)
            elif e == "*":
                self.entity['data']['entities']['s'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_subject_entity_type().")

    def update_subject_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_valency")
        self.entity['data']['valencies']['s'] = value

    def update_actant1_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a1'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a1'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant1_entity_type().")

    def update_actant2_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant2_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a2'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a2'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant2_entity_type().")

    def update_actant1_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a1'] = value

    def update_actant2_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a2'] = value


class ResourceEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_url(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        self.entity['data']['url'] = value


class ObjectEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class EventEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class LocationEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)


class EntityMapperFactory:
    def __init__(self):
        pass

    def make(self, entity_name, data_row):
        if 'ITerritory' == entity_name:
            return TerritoryEntityMapper(entity_name, data_row)
        elif 'IAction' == entity_name:
            return ActionEntityMapper(entity_name, data_row)
        elif 'IConcept' == entity_name:
            return ConceptEntityMapper(entity_name,data_row)
        elif 'IResource' == entity_name:
            return ResourceEntityMapper(entity_name, data_row)
        elif 'IObject' == entity_name:
            return ObjectEntityMapper(entity_name, data_row)
        elif 'IEvent' == entity_name:
            return EventEntityMapper(entity_name, data_row)
        elif 'ILocation' == entity_name:
            return LocationEntityMapper(entity_name, data_row)

        else:
            logger.warning(f"Unrecognized entity class in entity mapper. Is this right?")
            return EntityMapper(entity_name)





# CONTROL CLASS
class ParseController():

    def __init__(self, entity_list: [], keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}
        self.js_objects = []

        for e in self.entity_list:
            if 'texts' in e:
                self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'actions' in e:
                self.parsers[e] = ActionParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'concepts' in e:
                self.parsers[e] = ConceptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'resources' in e:
                self.parsers[e] = ResourceParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'manuscripts' in e:
                self.parsers[e] = ManuscriptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            else:
                self.logger.warning(f"Coming to basic Parser entity - strange '{e}' {type(e)}.")
                self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

    def load_json_objects(self):
        for name, p in self.parsers.items():
            self.js_objects = self.js_objects + p.js_objects

    def parse(self):
        for name, p in self.parsers.items():
            p.parse_rows()


# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],'propvalue':[],'propvalue_2nd':[],"dependent":[],"proptype":[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c]).strip()
            prop_type_candidate = str(prop_type_row.at[c]).strip()
            source_node_candidate = str(source_node_row.at[c]).strip()

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)


            # known instructions
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            elif 'propvalue' ==  instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate

                # test whether is its propvalue proper or dependent (=proptype is dynamic, value from another column)
                if prop_type.strip() == "":
                    self.oper_columns['dependent'].append(c)
                    continue # ignoring "dependent propvalue"

                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node} # source can be ignored, because the iProp object is sitting inside of it
                    self.oper_columns['propvalue'].append(c)

            elif 'propvalue_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue_2nd', 'type': prop_type, 'source':source_node} # source can NOT be ignored, it signals which existing iProp object will hold this iProp object
                    self.oper_columns['propvalue_2nd'].append(c)

            elif 'special' in instruction_candidate:
                # looks for custom functions registered by column name
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'special', 'type': prop_type, 'source':source_node}
                self.oper_columns['special'].append(c)

            elif 'proptype' in instruction_candidate:
                instruction  = {'operation':'proptype', 'type': prop_type, 'source':source_node}
                #logger.info(f"here ...{instruction_candidate} {c}")
                self.oper_columns['proptype'].append(c)

            elif 'dependent' in instruction_candidate:
                # ignore
                # the value is solved by another instruction
                instruction  = {'operation':'dependent', 'type': prop_type, 'source':source_node}

            elif 'inside' in instruction_candidate:
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'inside', 'target': prop_type_candidate}

                    self.oper_columns['inside'].append(c)

            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}, proptype: {len(self.oper_columns['proptype'])}, dependent:{len(self.oper_columns['dependent'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip


    def prepare_property(self):
        pass


    def make_row_object(self, data_row):
        class_name = table_to_entity[self.name]
        return self.EMP.make(class_name, data_row)

    def itemize_valuestring_for_multiples(self, value_with_multiples, origin="") -> []:
        values = []
        if isinstance(value_with_multiples,str):
            parsed_value = value_with_multiples.split('#')
            values =  [item.strip() for item in parsed_value]
        else:
            raise Exception(f"Expected value to be string. Got {type(value_with_multiples)}. {origin}")
        return values

    def parse_rows(self):
        self.logger.info(f"Starting to parse {self.name}.")

        if  self.prepared_table['label'].isnull().any():
            self.prepared_table  = self.prepared_table[self.prepared_table['label'].notna()]
            self.logger.info(f"Empty labels found in {self.name} table. (new entities added through parsing process). Adjustin - entities with empty labels not taken into account.")

        for key, row in self.prepared_table.iterrows():
            # self.logger.info(f"{self.name} Processing row {key}")
            entity_mapper = self.make_row_object(row.to_dict())

            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                # force string
                value = str(value)

                if operation['operation'] == 'inside' and value != '' and '?' not in name:
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']
                    entity_mapper.update_inside_field(name,value,operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")
                    for item in self.itemize_valuestring_for_multiples(value):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value), field_name = name)

                if operation['operation'] == 'propvalue_2nd' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    prop_source_name = operation['source']  # header_name,  we need concept_id
                    prop_source_id = self.parsing_instruction[prop_source_name]['type']

                    logger.info(f"{prop_source_name} : {prop_source_id} : {value}")

                    assert 'C' in prop_source_id, f"Trying to get to the concept of 1st level property to adress 2nd level property."

                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")

                    origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value)

                    for item in self.itemize_valuestring_for_multiples(value, origin=origin):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, prop_source = prop_source_id, origin = origin, field_name = name, prop_source_field = prop_source_name)

                if operation['operation'] == 'special' and value != "":
                    # logger.info(f"SPECIAL {operation} {value}")
                    func = getattr(self, 'special_'+name)
                    func(operation, value, entity_mapper)

            self.js_objects.append(entity_mapper.entity)

            # make audit records
            entity_mapper.create_audit_record()

            # break  DEV, checking parsing after first iteration



    def special_editor(self, operation, value, entity_mapper, field_name="special_editor"):
        pass


class TextParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # special methods for fields, which needs fully individual processing
    def special_edition_1(self, operation, value, entity_mapper, field_name="edition_1", ):
        # Parse this col. as "propvalue" - but you need to generate the target entities since they do not exist. How to do it: for any value, create an R entity with "label" = textual value in this col., "label language" = English, "status" = "approved", and "URL" = the hyperlink in the formula sitting on the textual value in this col. As usual, ignore NS and NA values (exact match) - do not import anything if the value is NA.
        # logger.info(f"Special edition1 running ...{operation} {value}")

        origin = operation['operation'] +" "+ operation['type']+ ">"+ ":"+field_name + str(value)
        prop_type = operation['type']

        # make rentity
        # logger.info(f"Rentity making {value}")
        if "|" in value:
            data = value.split("|")
            label = data[0]
            url = data[1]
        else:
            url = ""
            label = value
            # logger.warning(f"Expected char | signaling url after label. Got just {value}."+origin)

        rentity = entity_mapper.make_rentity(label, url, origin=origin)
        rentity['language'] = entity_mapper.enum_mapper['language']['English']
        entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)

        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = rentity['id'], origin = origin, field_name=field_name)

    def special_edition_2(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_2")

    def special_edition_3(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_3")

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        data = entity_mapper.data_row
        origin = operation['operation']+ " " + data['legacyId'] + ">"+ ":"+field_name +" "+ str(value)

        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)

        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = origin, field_name=field_name)

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]


        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass

class ActionParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


class ConceptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_wordnet_synset_id(self, operation, value, entity_mapper,field_name="special_wordnet_synset_id",):
        # wordnet_resource_id = R0067
        entity_mapper.hook_ref_object(ref_legacyID = "R0067", input_value = value, origin = operation['operation'] +">"+ ":"+field_name + " " +str(value))


class ManuscriptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        origin = operation['operation'] +">"+ ":"+field_name +" "+ str(value)

        data = entity_mapper.data_row
        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)


        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = operation['operation'] +">"+ ":"+field_name + str(value))

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        origin = origin + "LegacyId: "+ data['legacyId']
        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass


    def special_repository_label(self, operation, value, entity_mapper, field_name = "repository_label"):
        # For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = “approved”, entity logical type = “definite”, label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 “class” - C2646 “archive or library”, and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 “repository” - L in this col.

        origin = f"Making location '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            lentity = entity_mapper.make_lentity(label=value, legacyId="L_from_"+entity_mapper.data_row['legacyId'], origin=origin)

            lentity['language'] = entity_mapper.enum_mapper['language'][entity_mapper.data_row['repository_label_language']]

            prop_type_id = entity_mapper.get_entity_id("C0565", origin = origin)
            prop_value_id = entity_mapper.get_entity_id("C2646", origin = origin)
            prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
            lentity['props'].append(prop_object)

            # let the manuscript O own the location
            entity_mapper.hook_prop_object(prop_type = "C2645", input_value = lentity['id'], origin = origin)

            # create audit record
            entity_mapper.create_audit_record(entity_id=lentity['id'], object=lentity)


    # solves the method above
    def special_repository_label_language(self, operation, value, entity_mapper):
        pass

    def special_reproduction_online_url(self, operation, value, entity_mapper, field_name = "reproduction_online_url"):
        # If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = “approved”, label-language = “English”, url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 “digital reproduction” - the R entity here generated.

        origin = f"Making Resource entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value

            if "|" in value:
                data = value.split("|")
                label = data[0]
                url = data[1]
            else:
                url = ""
                label = value
                logger.warning(f"Expected char | signaling url after label. Got just {value}. "+origin)

            rentity = entity_mapper.make_rentity(label, url, origin=origin)
            entity_mapper.hook_prop_object(prop_type = "C1199", input_value = rentity['id'], origin = origin)

            # create audit record
            entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)


class ResourceParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

# Run

In [157]:
audits = [] # null audits
additional_entities = [] # null additional entities
# making empty table of additional entities
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])


logger.info(f"Start")
cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions'])
# cp = ParseController(entity_list=['texts'])
cp.parse()
logger.info(f"End")

2022-04-10 15:37:00,425 INFO Start
2022-04-10 15:37:00,432 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2022-04-10 15:37:00,435 INFO TEXTS 66 columns have been dropped (discard:66, unknown:0). Table now has 51 columns, inside:9,propvalue:14, special:6, proptype: 4, dependent:4. Originally 117 columns.
2022-04-10 15:37:00,439 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2022-04-10 15:37:00,441 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0). Table now has 31 columns, inside:8,propvalue:9, special:6, proptype: 4, dependent:4. Originally 66 columns.
2022-04-10 15:37:00,443 INFO RESOURCES Uncertain parsing instructions in 0 columns: .
2022-04-10 15:37:00,444 INFO RESOURCES 14 columns have been dropped (discard:14, unknown:0). Table now has 9 columns, inside:6,propvalue:2, special:1, proptype: 0, dependent:0. Originally 23 columns.
2022-04-10 15:37:00,448 INFO CONCEPTS Uncertain parsing instructions in 0 columns: .
2022-04-10 15:37:00,45

In [126]:
tables['props']

,id,type,type_id,value,value_id,original_field,origin,entityId,legacyId
0,02262ade-c394-48be-9e57-0f78caa64668,C0732,61f7b2fa-ef0b-4e88-82e0-301cfc5f8ebc,C0938,8dcb7734-bbb8-407f-905e-da9eaa6d0f83,language_id,propvalue>0:language_id:C0938,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,T1
1,3d2d2570-c102-4893-ab6c-99201e8a9e81,C0335,45f5e58a-15b5-429c-bd3f-fa4d1010ab72,C1134,c81abbdc-1e5f-46c5-a42e-d5a035a57302,genre_id,propvalue>0:genre_id:C1134,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,T1
2,2a95fcc5-be5a-4498-b0b4-8ec32493f825,C1178,50469a19-e7a7-436e-a3a0-8a7c29f3619d,C1138,1eec5151-ada6-473c-a5ad-5331d792e30c,milieu_of_provenance_id,propvalue>0:milieu_of_provenance_id:C1138,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,T1
3,0d27ca80-3dea-4dd3-876f-8b894741fb2a,C2642,898d2a37-d650-43e3-a10d-ac9cb00fdf51,205d7d74-6595-420c-abda-67cb6ec5c4c1,205d7d74-6595-420c-abda-67cb6ec5c4c1,creation_event_id,special T1>:creation_event_id E0001_T1,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,T1
4,2bb799eb-0177-479f-8c00-709f0846ea94,C1181,d635c057-7aea-46b7-93e4-12b19141dca3,M21,4a6368a5-63ef-4262-b0e0-26ddff371165,manuscript_witness_1,propvalue>0:manuscript_witness_1:M21,995e27c0-5c0b-4d08-ba8d-6fa538bbd5e1,T1
...,...,...,...,...,...,...,...,...,...
803,9d151752-06cb-49cf-b3ba-19d7c9b1f357,C0732,61f7b2fa-ef0b-4e88-82e0-301cfc5f8ebc,C0938,8dcb7734-bbb8-407f-905e-da9eaa6d0f83,language_id,propvalue>140:language_id:C0938,f2bfb8af-c2f7-49d0-8248-5d39dec6950a,T142
804,f86a7598-6835-4818-9c92-ef54245e3d73,C0335,45f5e58a-15b5-429c-bd3f-fa4d1010ab72,C1364,7f159ce6-1f20-4492-a161-2cfbbc947871,genre_id,propvalue>140:genre_id:C1364,f2bfb8af-c2f7-49d0-8248-5d39dec6950a,T142
805,5b87e982-88a4-4225-b401-067a9d238de7,C2642,898d2a37-d650-43e3-a10d-ac9cb00fdf51,c45d7955-b502-4a01-bb8b-ea8996a9b6b8,c45d7955-b502-4a01-bb8b-ea8996a9b6b8,creation_event_id,special T142>:creation_event_id E0141_T142,f2bfb8af-c2f7-49d0-8248-5d39dec6950a,T142
806,830be2e5-1171-4d90-abe7-d96560b9bcf8,C0732,61f7b2fa-ef0b-4e88-82e0-301cfc5f8ebc,C0938,8dcb7734-bbb8-407f-905e-da9eaa6d0f83,language_id,propvalue>141:language_id:C0938,fc5dfa74-593e-4184-8dc1-91de1785b207,T143


### Profiling

In [88]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [89]:
# %prun -s tottime profile()

In [90]:
# %lprun?

In [91]:
tables['events']

,id,value,origin,legacyId
0,4d21d66c-785d-4b5e-be3f-ab4f5ee4c8cc,Creation of ‘Process against Bernard Niort and...,special T1>:creation_event_id E0001_T1,E0001_T1
1,51eb3556-4cfc-4fe7-bc82-f14121c13fe2,Creation of ‘Sentences of William Arnold and S...,special T2>:creation_event_id E0002_T2,E0002_T2
2,c9d6543f-eba0-4979-83c8-b37a0d7b1e36,Creation of ‘Peter Seila’s Register of Penances’,special T3>:creation_event_id E0003_T3,E0003_T3
3,eef418e1-3885-4a4c-8eba-613f9bc496fd,Creation of ‘Register FFF of the Carcassonne i...,special T4>:creation_event_id E0004_T4,E0004_T4
4,302d7d9f-bc42-4e8f-af3f-d6d8e5df5f9c,Creation of ‘Confirmation of depositions befor...,special T5>:creation_event_id E0005_T5,E0005_T5
...,...,...,...,...
276,3f73a7fc-684b-453f-9c89-a942a081f153,"Creation of St Paul-im-Lavanttal, Stift Sankt ...",special>:creation_event_id E0135_M153,E0135_M153
277,62bba972-717b-4da7-b842-251db5d88605,"Creation of Roma, Biblioteca Casanatense, ms. ...",special>:creation_event_id E0136_M154,E0136_M154
278,249572c2-6d5a-4d9d-9bb8-e9a50a9ccd5f,"Creation of Roma, Archivio Generale dell’Ordin...",special>:creation_event_id E0137_M155,E0137_M155
279,e4212384-7f47-49d1-9158-1391c536dd8b,"Creation of Wien, Österreichische Nationalbibl...",special>:creation_event_id E0138_M156,E0138_M156


In [92]:
# what is this d72b2bee-47b4-4abc-8c88-793c72ab676d ?

pd.DataFrame(additional_entities)

,class,id,label,language,detail,data,props,notes,status,references,legacyId
0,E,4d21d66c-785d-4b5e-be3f-ab4f5ee4c8cc,Creation of ‘Process against Bernard Niort and...,eng,,{'logicalType': '1'},"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [development] 2022-04-10 10:28:3...,1,[],E0001_T1
1,V,b2953607-b2b8-4583-87e5-8415ac445b70,1234,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
2,V,62c54952-e825-4d69-98d2-59edaa9c0344,1234,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
3,V,aee2ac76-9391-4dfc-9322-edc006bd1906,34r-50r,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
4,R,7a182785-c4b9-4b97-ab47-a51a74eb4826,"Douais, 1900",eng,,{'url': 'https://www.zotero.org/groups/446972/...,[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
1940,V,27edc0e3-223d-493b-b964-51a4ce499b57,???,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
1941,V,b273b99d-6b86-4316-b3c1-2880c8e00314,???,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
1942,V,5f895178-b07f-4fd3-a541-93e18ad0dc8b,???,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN
1943,V,37b3425b-4e21-4115-a66f-08825586c36b,???,,,{'logicalType': '4'},[],[Import batch [development] 2022-04-10 10:28:3...,1,[],NaN


### Last checks

In [93]:
print(json.dumps(cp.parsers['actions'].js_objects[0]))

{"class": "A", "id": "1da83bae-3e51-465e-8c1f-14e45ea4a501", "legacyId": "A0168", "label": "abiurationem recepit", "language": "lat", "detail": "received abjuration (from sb)", "data": {"entities": {"a1": ["P", "G"], "a2": ["NULL"], "s": ["P", "G"]}, "valencies": {"a1": "\"a\" + 6 | 2", "a2": "NULL", "s": "1"}}, "props": [{"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "848af435-169c-4941-b194-5f9dc9cb0712", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "665bb74c-4d5a-4c3f-afc4-2a9c6e142749", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}, "value": {"id": "da4a2ab0-9053-4a48-bf75-250e0220aef4", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}}, {"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "388eabb1-4a95-46ee-8460-4edd5539c87e", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "f6b8

In [94]:
logger.info(f"There are {len(additional_entities)} additionally created entities (e.g. values, resources ...).")

2022-04-10 10:32:07,784 INFO There are 1945 additionally created entities (e.g. values, resources ...).


# Output the parsed data

In [150]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "root",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [158]:
# connects outcomes from the individual parsers
cp.load_json_objects()

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))


# write the audits json.
with open('../datasets/all-parsed/audits.json','w', encoding='utf-8') as f:
    json.dump(audits, f)


logger.info("END")

2022-04-10 15:51:41,064 INFO END


In [97]:
# Log tables google sheet
# https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE/edit#gid=0


d.open_gsheet("logtables","https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE")

d.write_df_to_gsheet("logtables", "values", tables["values"])
d.write_df_to_gsheet("logtables", "resources", tables["resources"])
d.write_df_to_gsheet("logtables", "locations", tables["locations"])
d.write_df_to_gsheet("logtables", "events", tables["events"])


20 2022-04-10 10:32:11 : Opening gsheet handle logtables
20 2022-04-10 10:32:11 : Trying to write target dataset 'values' to gsheet handle: logtables
20 2022-04-10 10:32:11 : Using existing output worksheet
20 2022-04-10 10:32:13 : Successfully written to gsheet handle: logtables
20 2022-04-10 10:32:13 : Pickle not written, the output_tables dir probably does not exists.
20 2022-04-10 10:32:13 : Trying to write target dataset 'resources' to gsheet handle: logtables
20 2022-04-10 10:32:13 : Using existing output worksheet
20 2022-04-10 10:32:15 : Successfully written to gsheet handle: logtables
20 2022-04-10 10:32:15 : Pickle not written, the output_tables dir probably does not exists.
20 2022-04-10 10:32:15 : Trying to write target dataset 'locations' to gsheet handle: logtables
20 2022-04-10 10:32:15 : Using existing output worksheet
20 2022-04-10 10:32:16 : Successfully written to gsheet handle: logtables
20 2022-04-10 10:32:16 : Pickle not written, the output_tables dir probably doe